In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

C:\Users\alexanjs\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv('./data/raw_results.csv')
df.head()

,Date,Round,Region,Winning Seed,Winner,Winning Score,Losing Seed,Loser,Losing Score,Overtime
0,3/14/85,Round of 64,East,1,Georgetown,68,16,Lehigh,43,NaN
1,3/14/85,Round of 64,East,4,"Loyola, Illinois",59,13,Iona,58,NaN
2,3/14/85,Round of 64,East,5,Southern Methodist,85,12,Old Dominion,68,NaN
3,3/14/85,Round of 64,East,8,Temple,60,9,Virginia Tech,57,NaN
4,3/14/85,Round of 64,Midwest,1,Oklahoma,96,16,North Carolina A&T,83,NaN


In [3]:

df.dtypes


Date             object
Round            object
Region           object
Winning Seed      int64
Winner           object
Winning Score     int64
Losing Seed       int64
Loser            object
Losing Score      int64
Overtime         object
dtype: object

In [4]:
df['Year'] = df['Date'].astype(str).str[-2:]

In [5]:
df.head()

,Date,Round,Region,Winning Seed,Winner,Winning Score,Losing Seed,Loser,Losing Score,Overtime,Year
0,3/14/85,Round of 64,East,1,Georgetown,68,16,Lehigh,43,NaN,85
1,3/14/85,Round of 64,East,4,"Loyola, Illinois",59,13,Iona,58,NaN,85
2,3/14/85,Round of 64,East,5,Southern Methodist,85,12,Old Dominion,68,NaN,85
3,3/14/85,Round of 64,East,8,Temple,60,9,Virginia Tech,57,NaN,85
4,3/14/85,Round of 64,Midwest,1,Oklahoma,96,16,North Carolina A&T,83,NaN,85


In [6]:
dictionary = {'National Championship': 0, 
              'National Semifinals': 1, 
              'Elite Eight': 2, 
              'Sweet Sixteen': 3, 
              'Round of 32': 4, 
              'Round of 64': 5, 
              'Opening Round': 6}

In [7]:
df['RoundNumber'] = df.Round.map(lambda x: dictionary[x])

In [8]:
df.tail()

,Date,Round,Region,Winning Seed,Winner,Winning Score,Losing Seed,Loser,Losing Score,Overtime,Year,RoundNumber
2045,3/27/16,Elite Eight,East,1,North Carolina,88,6,Notre Dame,74,NaN,16,2
2046,3/27/16,Elite Eight,Midwest,10,Syracuse,68,1,Virginia,62,NaN,16,2
2047,4/2/16,National Semifinals,NaN,1,North Carolina,83,10,Syracuse,66,NaN,16,1
2048,4/2/16,National Semifinals,NaN,2,Villanova,95,2,Oklahoma,51,NaN,16,1
2049,4/4/16,National Championship,NaN,2,Villanova,77,1,North Carolina,74,NaN,16,0


In [9]:
df_new = df[['Winner', 'Loser', 'Year', 'RoundNumber']]

In [10]:
df_new.head()

,Winner,Loser,Year,RoundNumber
0,Georgetown,Lehigh,85,5
1,"Loyola, Illinois",Iona,85,5
2,Southern Methodist,Old Dominion,85,5
3,Temple,Virginia Tech,85,5
4,Oklahoma,North Carolina A&T,85,5


In [11]:
list_of_losers = []

for index, row in df_new.iterrows():
    loser = row.Loser
    rd = row.RoundNumber
    year = row.Year
    list_of_losers.append({'School': loser, 'Round': rd, 'Year': year})
    if rd is 0: 
        list_of_losers.append({'School': row.Winner, 'Round': rd-1, 'Year': year})
    
targets = pd.DataFrame(list_of_losers)
targets['Round'] = targets['Round'] + 1
targets.head()
targets['School'] = targets['School'].astype(str)
targets['Year'] = targets['Year'].astype('int64')
targets[targets['School'] == 'Duke']
targets.dtypes

Round      int64
School    object
Year       int64
dtype: object

In [12]:
targets.to_csv('./data/round_team_year.csv')

In [13]:
stats = pd.read_excel('./data/2012stats.xlsx', skiprows=1)

In [14]:
stats.head()

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Air Force,29,13,16,0.448,-0.45,2.14,3,11,...,362,537,0.674,178,767,403,194,90,371,491
1,2,Akron,34,22,12,0.647,6.33,0.72,13,3,...,513,741,0.692,398,1098,475,247,141,480,616
2,3,Alabama A&M,28,7,21,0.250,-19.33,-10.79,5,13,...,359,555,0.647,316,849,314,178,119,463,617
3,4,Alabama-Birmingham,31,15,16,0.484,2.61,3.64,9,7,...,388,566,0.686,350,997,352,136,110,439,494
4,5,Alabama State,31,12,19,0.387,-15.79,-9.96,9,9,...,354,606,0.584,400,982,357,236,95,470,614


In [15]:
stats=stats.drop('Rk', axis=1)

In [16]:
stats['Year'] = 12

stats['School'] = stats['School'].astype(str)

In [17]:
stats.head()

,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year
0,Air Force,29,13,16,0.448,-0.45,2.14,3,11,9,...,537,0.674,178,767,403,194,90,371,491,12
1,Akron,34,22,12,0.647,6.33,0.72,13,3,13,...,741,0.692,398,1098,475,247,141,480,616,12
2,Alabama A&M,28,7,21,0.250,-19.33,-10.79,5,13,6,...,555,0.647,316,849,314,178,119,463,617,12
3,Alabama-Birmingham,31,15,16,0.484,2.61,3.64,9,7,8,...,566,0.686,350,997,352,136,110,439,494,12
4,Alabama State,31,12,19,0.387,-15.79,-9.96,9,9,7,...,606,0.584,400,982,357,236,95,470,614,12


In [18]:
stats['School'] = stats['School'].str.replace('NCAA', '')
stats['School'] = stats['School'].str.strip()

In [19]:
table = pd.merge(stats, targets, on=['School', 'Year'])

In [20]:
table[table['School'] == 'Duke']

,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year,Round
10,Duke,34,27,7,0.794,17.05,8.4,13,3,13,...,0.701,406,1152,423,210,136,415,626,12,6


In [21]:
table.shape

(60, 35)

In [22]:
table

,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year,Round
0,Alabama,33,21,12,0.636,13.76,7.18,9,7,12,...,0.711,352,1066,402,239,137,423,607,12,6
1,Baylor,38,30,8,0.789,16.54,8.03,12,6,14,...,0.754,470,1280,570,286,178,522,674,12,3
2,Belmont,35,27,8,0.771,11.81,-1.22,16,2,13,...,0.734,401,1151,606,255,78,430,631,12,6
3,Brigham Young,35,26,9,0.743,11.25,1.55,12,4,13,...,0.700,381,1223,599,296,154,473,637,12,6
4,Cincinnati,37,26,11,0.703,12.93,6.28,12,6,14,...,0.644,489,1225,479,298,170,403,570,12,4
5,Colorado State,32,20,12,0.625,6.58,5.23,8,6,14,...,0.768,266,939,389,172,60,408,606,12,6
6,Colorado,36,24,12,0.667,7.91,5.73,11,7,14,...,0.691,351,1192,422,210,117,468,614,12,5
7,Connecticut,34,20,14,0.588,12.27,8.07,8,10,13,...,0.662,415,1151,457,199,233,418,492,12,6
8,Creighton,35,29,6,0.829,12.92,3.41,14,4,14,...,0.726,330,1163,616,169,82,421,592,12,5
9,Davidson,33,25,8,0.758,7.26,-1.68,16,2,13,...,0.763,415,1167,455,184,116,388,673,12,6


In [23]:
lgr = LogisticRegression()

features = table.drop(['School', 'Year', 'Round', 'Unnamed: 16', 'G'], axis=1)
targets = table.Round
features = (features - features.mean())/features.std()

features.describe()

,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,L.2,W.3,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
count,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,...,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01
mean,1.889692e-16,-2.918961e-16,9.820848e-16,-1.591320e-16,-2.016905e-16,2.430926e-16,1.369275e-16,-3.724162e-16,-9.621933e-17,1.056447e-16,...,7.845576e-16,1.258253e-16,1.300811e-14,6.291264e-17,1.850372e-17,3.460195e-16,-2.553513e-16,1.225871e-16,-1.332268e-16,4.921989e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.525157e+00,-2.384936e+00,-3.091885e+00,-2.622726e+00,-2.161140e+00,-2.114266e+00,-1.973957e+00,-2.595755e+00,-1.330839e+00,-1.730747e+00,...,-2.364768e+00,-2.475460e+00,-2.194030e+00,-2.184963e+00,-1.837140e+00,-1.619304e+00,-1.512458e+00,-1.478851e+00,-2.408921e+00,-2.193139e+00
25%,-4.425533e-01,-4.157227e-01,-6.697438e-01,-6.439862e-01,-8.019896e-01,-8.866276e-01,-6.865936e-01,-4.386179e-01,-6.920365e-01,-8.795597e-01,...,-5.913684e-01,-6.548451e-01,-6.722065e-01,-6.028268e-01,-4.821993e-01,-8.263460e-01,-7.697330e-01,-6.598100e-01,-5.956877e-01,-6.974306e-01
50%,7.809764e-02,-8.752057e-02,1.041120e-01,7.791417e-02,3.030293e-01,-6.820212e-02,-4.291210e-02,-7.190458e-03,-5.323358e-02,-2.837289e-02,...,6.792745e-02,2.152869e-02,-4.557332e-02,-6.535460e-02,-6.774701e-02,-1.933705e-01,-1.308858e-01,-1.448443e-01,-1.283595e-01,1.014135e-01
75%,3.384231e-01,5.688837e-01,5.734542e-01,7.191000e-01,8.129944e-01,7.502233e-01,6.007694e-01,8.556645e-01,5.855693e-01,3.972205e-01,...,5.826718e-01,5.453369e-01,5.566456e-01,7.824889e-01,2.709880e-01,1.030846e+00,7.416861e-01,3.750258e-01,5.866527e-01,5.815698e-01
max,3.202003e+00,3.194501e+00,2.417298e+00,2.057066e+00,1.315018e+00,1.977862e+00,2.317253e+00,2.149947e+00,3.140781e+00,2.525188e+00,...,2.486521e+00,3.169464e+00,1.842464e+00,2.614436e+00,3.494949e+00,2.345487e+00,2.871177e+00,4.092588e+00,2.376520e+00,2.107022e+00


In [24]:
from sklearn.model_selection import cross_validate

lgr = LogisticRegression()

In [25]:
lgr.fit(features[:-20], targets[:-20])
lgr.score(features[-20:], targets[-20:])

0.55

In [26]:
lgr.predict(features[table['School'] == 'Purdue'])[0]

4

In [27]:
import numpy as np
from sklearn.metrics import mean_squared_error

features_train = train.drop(['School', 'Year', 'Round', 'Unnamed: 16', 'G'], axis=1)
features_test = test.drop(['School', 'Year', 'Round', 'Unnamed: 16', 'G'], axis=1)

targets_train = train.Round
targets_test = test.Round

features_train = (features_train - features_train.mean())/features_train.std()
features_test = (features_test - features_test.mean())/features_test.std()

NameError: name 'train' is not defined

In [ ]:
grid = {'max_depth': np.arange(1, 26)}

classifier = DecisionTreeClassifier()
classifierCV = GridSearchCV(classifier, param_grid=grid, return_train_score=True)
classifierCV.fit(features_train, targets_train)


In [ ]:

print(classifierCV.best_params_, classifierCV.best_score_)
print("Test Score", classifierCV.score(features_test, targets_test))
print("Train Score", classifierCV.score(features_train, targets_train))

target_predicted = classifierCV.predict(features_test)
RMSE_test = np.sqrt(mean_squared_error(targets_test, target_predicted))
print("RMSE", RMSE_test)

In [28]:
df_test = table.sample(frac=.2, replace=False, random_state=0).copy()
df_train = table.drop(df_test.index).copy()

features_train = df_train.drop(['School', 'Year', 'Round', 'Unnamed: 16', 'G'], axis=1)
features_test = df_test.drop(['School', 'Year', 'Round', 'Unnamed: 16', 'G'], axis=1)

targets_train = df_train.Round
targets_test = df_test.Round

In [30]:
rf = RandomForestClassifier(n_estimators=2000, n_jobs=-1, oob_score=True, max_depth=7)
rf.fit(features_train, targets_train)
target_predicted = rf.predict(features_test)
RMSE_test = np.sqrt(mean_squared_error(targets_test, target_predicted))

print("Test Score", rf.score(features_test, targets_test))
print("Train Score", rf.score(features_train, targets_train))
print(RMSE_test)
print(rf.oob_score_)


Test Score 0.5833333333333334
Train Score 1.0
0.816496580927726
0.4791666666666667
